##CONJUNTO DE DATOS DEL PROCESO IMPREGNACIÓN PARA EL CONTROLAR DE DOSIFICACIÓN DE RESINA IMPREGNADA EN PAPEL MEDIANTE LA SEPARACIÓN DE LOS DOS RODILLOS GAP.

El proposito principal de este dataset es predecir el valor del GAP entre dos rodillos encargados de controlar la dosificación de resina en un rollo de papel que es desbobinado automaticamente. En este dataset existen dos variables objectivo del estudio, uno es GAP LO(mm) y otro es GAP LC(mm).

In [1]:
!pip install mglearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.4/581.4 kB 4.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt                                                                                      #libreria para generación de graficos en 2D
import mglearn as mg
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split                                                                 #libreria para dividir conjunto de datos en train y test.
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.metrics import accuracy_score                                                    #libreria para usar matriz de confusión, calcular la presición, recall y exactitud.
from sklearn.linear_model import Lasso
import seaborn as sns
from scipy import stats
from matplotlib.patheffects import withStroke
import itertools
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder

##ENTENDIMIENTO DE LOS DATOS

**LEEMOS EL DATASET QUE CONTIENE LOS DATOS DEL PROCESO DE LA IMPREGNADORA DE PAPEL.**

In [3]:
proyecto=pd.read_csv('https://raw.githubusercontent.com/aristidesuao/proyectoAP/main/proyectoAP.csv',delimiter=';') #archivo csv que los datos del proceso gap de la impregnadora de papel.

df=pd.DataFrame(proyecto) #se crea dataframe

el archivo CSV está en un repositorio de github.

In [4]:
df.head() #se muestra el dataframe

,DISENO,Gramaje,PLANTA,GAP LO(mm),GAP LT(mm),TEMPERATURA HORNO 1(C),TEMPERATURA HORNO 2(C),TEMPERATURA HORNO 3(C),TEMPERATURA HORNO 4(C),TEMPERATURA HORNO 5(C),...,PESO IMPREGNADO LO1,PESO IMPREGNADO LC,PESO IMPREGNADO LT1,VOLATILES LO2,VOLATILES LC2,VOLATILES LT2,FLUIDEZ,ESTADO B LO3,ESTADO B LC3,ESTADO B LT3
0,Nuez,60,FF1,"0,09","0,087",126,126,126,217,220,...,"154,6","155,3","155,6","5,7","4,9","5,6","4,1",36,"59,1","42,5"
1,Nuez,60,FF1,"0,09","0,087",126,126,126,217,220,...,"154,6","155,3","155,6","5,7","4,9","5,6","4,1",36,"59,1","42,5"
2,Nuez,60,BP1,"0,09","0,087",130,130,130,214,220,...,"153,7","154,2","151,8","6,2","5,8","6,1",5,"22,6","38,8","30,4"
3,Nuez,60,BP1,"0,09","0,087",130,130,130,214,220,...,"156,6","153,7",154,"5,7","5,2","5,8",5,"24,7","44,1","29,4"
4,Nuez,60,BP1,"0,09","0,087",130,130,130,214,220,...,"148,9","148,9","147,6","5,6",5,"5,7","4,1","20,9",37,"21,5"


In [5]:
print("El número de registros es: ", df.shape[0])
print("El número de atributos es: ", df.shape[1])

El número de registros es:  172
El número de atributos es:  28


El dataset contiene en total 172 registro y 28 columnas.

In [6]:
df.columns

Index(['DISENO', 'Gramaje ', 'PLANTA', 'GAP LO(mm)', 'GAP LT(mm)',
       ' TEMPERATURA HORNO 1(C)', ' TEMPERATURA HORNO 2(C)',
       ' TEMPERATURA HORNO 3(C)', ' TEMPERATURA HORNO 4(C)',
       ' TEMPERATURA HORNO 5(C)', ' TEMPERATURA HORNO 6(C)',
       'VELOCIDAD VENTILADOR HORNO 1(rpm)',
       'VELOCIDAD VENTILADOR HORNO 2(rpm)',
       'VELOCIDAD VENTILADOR HORNO 3(rpm)',
       'VELOCIDAD VENTILADOR HORNO 4 (rpm)',
       'VELOCIDAD VENTILADOR HORNO 5 (rpm)',
       'VELOCIDAD VENTILADOR HORNO 6 (rpm)', ' VELOCIDAD DE LA LINEA m/min',
       'PESO IMPREGNADO LO1', 'PESO IMPREGNADO LC', 'PESO IMPREGNADO LT1',
       'VOLATILES LO2', 'VOLATILES LC2', ' VOLATILES LT2', 'FLUIDEZ',
       'ESTADO B LO3', 'ESTADO B LC3', 'ESTADO B LT3'],
      dtype='object')

#Descripción de cada target

GAP LO(mm): separación en milimetros del lado del operacio entre los dos rodillos encargados de controlar la dosificación de resina en el papel. Este variable es un target. es una variable tipo numerica float.

GAP LC(mm): separación en milimetros del lado contrario del operacio entre los dos rodillos encargados de controlar la dosificación de resina en el papel. Este variable es un target. es una variable tipo numerica float.

#Descripción de cada caracteristica:

DISEÑO:es el tipo de papel a impregnar. Es una variable categorica.

Gramaje:peso por metro cuadrado del papel. Es una variable tipo numerica entero.

PLANTA:es la planta laminadora de papel para el cual se fabrica el papel impregnado. Es una variable categorica

TEMPERATURA HORNO DEL 1: Es la temperatura del horno numero uno de la planta de impregnación. Está en grados celsius. Es una variable tipo numerica float.

TEMPERATURA HORNO DEL 2: Es la temperatura del horno numero dos de la planta de impregnación. Está en grados celsius. Es una variable tipo numerica float.

TEMPERATURA HORNO DEL 3: Es la temperatura del horno numero tres de la planta de impregnación. Está en grados celsius. Es una variable tipo numerica float.

TEMPERATURA HORNO DEL 4: Es la temperatura del horno numero cuatro de la planta de impregnación. Está en grados celsius. Es una variable tipo numerica float.

TEMPERATURA HORNO DEL 5: Es la temperatura del horno numero cinco de la planta de impregnación. Está en grados celsius. Es una variable tipo numerica float.

TEMPERATURA HORNO DEL 6: Es la temperatura del horno numero 6 de la planta de impregnación. Está en grados celsius. Es una variable tipo numerica float.

VELOCIDAD VENTILADOR HORNO del 1: es la velocidad del ventilador del horno numero uno donde se seca la resina impregnada en el papel. Está en rpm. Es una variable tipo numerica float.

VELOCIDAD VENTILADOR HORNO del 2: es la velocidad del ventilador del horno numero dos donde se seca la resina impregnada en el papel. Está en rpm. Es una variable tipo numerica float.

VELOCIDAD VENTILADOR HORNO del 3: es la velocidad del ventilador del horno numero tres donde se seca la resina impregnada en el papel. Está en rpm. Es una variable tipo numerica float.

VELOCIDAD VENTILADOR HORNO del 4: es la velocidad del ventilador del horno numero cuatro donde se seca la resina impregnada en el papel. Está en rpm. Es una variable tipo numerica float.

VELOCIDAD VENTILADOR HORNO del 5: es la velocidad del ventilador del horno numero cinco donde se seca la resina impregnada en el papel. Está en rpm. Es una variable tipo numerica float.

VELOCIDAD VENTILADOR HORNO del 6: es la velocidad del ventilador del horno numero seis donde se seca la resina impregnada en el papel. Está en rpm. Es una variable tipo numerica float.

VELOCIDAD DE LA LINEA es la velocidad de los rodillos que generan la tension necesaria para que el papel se transporte desde la desbobinadora hasta la estación de corte de papel. Es una variable tipo numerica float.

PESO IMPREGNADO LO1(lado operación): es la suma del papel base, la resina, los aditivos y la humedad del papel. Es la muestra tomada del lado izquierdo del papel seco. Es una variable tipo numerica float.

PESO IMPREGNADO LC(lado centro de operación): es la suma del papel base, la resina, los aditivos y la humedad del papel. Es la muestra tomada en todo el centro del papel seco. Es una variable tipo numerica float.

PESO IMPREGNADO LT1(lado contrario de operación): es la suma del papel base, la resina, los aditivos y la humedad del papel. Es la muestra tomada del lado derecho del papel seco. Es una variable tipo numerica float.

VOLATILES LO2(lado operación): Los volátiles es la cantidad de agua que le queda al papel luego de tomar una muestra del lado izquierdo del papel seco y llevarla a 160° C durante 5 min. Es una variable tipo numerica float.

VOLATILES LC2(lado centro de operación): Los volátiles es la cantidad de agua que le queda al papel luego de tomar una muestra del centro del papel seco y llevarla a 160° C durante 5 min. Es una variable tipo numerica float.

VOLATILES LT2(lado contario de operación): Los volátiles es la cantidad de agua que le queda al papel luego de tomar una muestra del lado derecho del papel seco y llevarla a 160° C durante 5 min. Es una variable tipo numerica float.

FLUIDEZ: es una prueba que muestra si el papel fluye dentro de las plantas de laminacion. Es una variable tipo numerica float.

ESTADO B LO3(lado operación): Es un análisis que se hace para determinar el grado de curado de una muestra que se toma del lado izquierdo del papel seco. Es una variable tipo numerica float.

ESTADO B LC3(lado centro operación): Es un análisis que se hace para determinar el grado de curado de una muestra que se toma del lado centro del papel seco. Es una variable tipo numerica float.

ESTADO B LT3(lado contrario operación): Es un análisis que se hace para determinar el grado de curado de una muestra que se toma del lado derecho del papel seco. Es una variable tipo numerica float.

#RENOMBRAMOS ALGUNAS COLUMNAS

In [7]:
df=df.rename(columns={'Gramaje ':'Gramaje',' TEMPERATURA HORNO 1(C)':'Temp Horno 1',' TEMPERATURA HORNO 2(C)':'Temp Horno 2',' TEMPERATURA HORNO 3(C)':'Temp Horno 3'
                      ,' TEMPERATURA HORNO 4(C)':'Temp Horno 4',' TEMPERATURA HORNO 5(C)':'Temp Horno 5',' TEMPERATURA HORNO 6(C)':'Temp Horno 6',
                      'VELOCIDAD VENTILADOR HORNO 1(rpm)':'Vel Vent Horno 1','VELOCIDAD VENTILADOR HORNO 2(rpm)':'Vel Vent Horno 2','VELOCIDAD VENTILADOR HORNO 3(rpm)':'Vel Vent Horno 3',
                      'VELOCIDAD VENTILADOR HORNO 4 (rpm)':'Vel Vent Horno 4','VELOCIDAD VENTILADOR HORNO 5 (rpm)':'Vel Vent Horno 5','VELOCIDAD VENTILADOR HORNO 6 (rpm)':'Vel Vent Horno 6'
                       ,' VELOCIDAD DE LA LINEA m/min':'Vel Linea',' VOLATILES LT2':'VOLATILES LT2'})

In [8]:
df.columns

Index(['DISENO', 'Gramaje', 'PLANTA', 'GAP LO(mm)', 'GAP LT(mm)',
       'Temp Horno 1', 'Temp Horno 2', 'Temp Horno 3', 'Temp Horno 4',
       'Temp Horno 5', 'Temp Horno 6', 'Vel Vent Horno 1', 'Vel Vent Horno 2',
       'Vel Vent Horno 3', 'Vel Vent Horno 4', 'Vel Vent Horno 5',
       'Vel Vent Horno 6', 'Vel Linea', 'PESO IMPREGNADO LO1',
       'PESO IMPREGNADO LC', 'PESO IMPREGNADO LT1', 'VOLATILES LO2',
       'VOLATILES LC2', 'VOLATILES LT2', 'FLUIDEZ', 'ESTADO B LO3',
       'ESTADO B LC3', 'ESTADO B LT3'],
      dtype='object')

con el método info podemos buscar valores nulos en alguna de las variables del dataset:

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   DISENO               172 non-null    object
 1   Gramaje              172 non-null    int64 
 2   PLANTA               172 non-null    object
 3   GAP LO(mm)           172 non-null    object
 4   GAP LT(mm)           172 non-null    object
 5   Temp Horno 1         172 non-null    int64 
 6   Temp Horno 2         172 non-null    int64 
 7   Temp Horno 3         172 non-null    int64 
 8   Temp Horno 4         172 non-null    int64 
 9   Temp Horno 5         172 non-null    int64 
 10  Temp Horno 6         172 non-null    int64 
 11  Vel Vent Horno 1     172 non-null    int64 
 12  Vel Vent Horno 2     172 non-null    int64 
 13  Vel Vent Horno 3     172 non-null    int64 
 14  Vel Vent Horno 4     172 non-null    int64 
 15  Vel Vent Horno 5     172 non-null    int64 
 16  Vel Vent

Teniendo en cuenta que el dataset contiene 172 registros se puede notar que las ultimas 3 variables hay solo 171, lo que quiere decir que existe un valor nulo en cada una de estas variables. Además, también se puede observar que las variablres 3,4,18,19,20,21,22,23,24,25,26,27 son tipo object, cuando realmente deberian ser tipo numerica. Esto es indicio de que existe bien un valor nulo o un valor no numerico.

Se procede a convertir las columnas anteriormente mencionadas a valor float.

In [10]:
try:
  df=df.astype({'GAP LO(mm)':'float64','GAP LT(mm)':'float64','PESO IMPREGNADO LO1':'float64',
                'PESO IMPREGNADO LC':'float64','PESO IMPREGNADO LT1':'float64','VOLATILES LO2':'float64',
                'VOLATILES LC2':'float64','VOLATILES LT2':'float64','FLUIDEZ':'float64','ESTADO B LO3':'float64',
                'ESTADO B LC3':'float64','ESTADO B LT3':'float64'})
except ValueError as e:
    print(e)

could not convert string to float: '0,09'


El error anterior se debe a que el decimal de los registros de estas variables están separadas por comas y no por puntos.

In [11]:
df=df.replace(",",".",regex=True) # se reemplaza cada coma por punto en los registro del dataset.

Se procede a convertir las columnas anteriormente mencionadas a valor float.

In [12]:
try:
  df=df.astype({'GAP LO(mm)':'float64','GAP LT(mm)':'float64','PESO IMPREGNADO LO1':'float64',
                'PESO IMPREGNADO LC':'float64','PESO IMPREGNADO LT1':'float64','VOLATILES LO2':'float64',
                'VOLATILES LC2':'float64','VOLATILES LT2':'float64','FLUIDEZ':'float64','ESTADO B LO3':'float64',
                'ESTADO B LC3':'float64','ESTADO B LT3':'float64'})
except ValueError as e:
    print(e)

could not convert string to float: '-'


El error anterior se debe a que un registro de las variables anteriores contiene el caracter "-". Es por esto que las variables son tipo object y no numerica. Se procede a reemplazar el caracter "-" por la moda de cada variable.

In [16]:
columns=['GAP LO(mm)','GAP LT(mm)','PESO IMPREGNADO LO1',
                'PESO IMPREGNADO LC','PESO IMPREGNADO LT1','VOLATILES LO2',
                'VOLATILES LC2','VOLATILES LT2','FLUIDEZ','ESTADO B LO3',
                'ESTADO B LC3','ESTADO B LT3']

for i in range(0,len(columns)):
   df[columns[i]]=df[columns[i]].replace("-",df[columns[i]].mode()[0],regex=True)

Se procede a convertir las columnas anteriormente mencionadas a valor float.

In [17]:
try:
  df=df.astype({'GAP LO(mm)':'float64','GAP LT(mm)':'float64','PESO IMPREGNADO LO1':'float64',
                'PESO IMPREGNADO LC':'float64','PESO IMPREGNADO LT1':'float64','VOLATILES LO2':'float64',
                'VOLATILES LC2':'float64','VOLATILES LT2':'float64','FLUIDEZ':'float64','ESTADO B LO3':'float64',
                'ESTADO B LC3':'float64','ESTADO B LT3':'float64'})
except ValueError as e:
    print(e)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   DISENO               172 non-null    object 
 1   Gramaje              172 non-null    int64  
 2   PLANTA               172 non-null    object 
 3   GAP LO(mm)           172 non-null    float64
 4   GAP LT(mm)           172 non-null    float64
 5   Temp Horno 1         172 non-null    int64  
 6   Temp Horno 2         172 non-null    int64  
 7   Temp Horno 3         172 non-null    int64  
 8   Temp Horno 4         172 non-null    int64  
 9   Temp Horno 5         172 non-null    int64  
 10  Temp Horno 6         172 non-null    int64  
 11  Vel Vent Horno 1     172 non-null    int64  
 12  Vel Vent Horno 2     172 non-null    int64  
 13  Vel Vent Horno 3     172 non-null    int64  
 14  Vel Vent Horno 4     172 non-null    int64  
 15  Vel Vent Horno 5     172 non-null    int

Se observa que todavía las variables del 25 al 27 tiene 171 registros, cuando debería ser 172.

Verificamos que existan datos nulos

In [19]:
max(df.isnull().sum(axis=1))

3

Existen tres datos nulos en el datos. Se necesita saber en que variables están esos datos nulos.

In [20]:
df.isnull().sum()

DISENO                 0
Gramaje                0
PLANTA                 0
GAP LO(mm)             0
GAP LT(mm)             0
Temp Horno 1           0
Temp Horno 2           0
Temp Horno 3           0
Temp Horno 4           0
Temp Horno 5           0
Temp Horno 6           0
Vel Vent Horno 1       0
Vel Vent Horno 2       0
Vel Vent Horno 3       0
Vel Vent Horno 4       0
Vel Vent Horno 5       0
Vel Vent Horno 6       0
Vel Linea              0
PESO IMPREGNADO LO1    0
PESO IMPREGNADO LC     0
PESO IMPREGNADO LT1    0
VOLATILES LO2          0
VOLATILES LC2          0
VOLATILES LT2          0
FLUIDEZ                0
ESTADO B LO3           1
ESTADO B LC3           1
ESTADO B LT3           1
dtype: int64

Las caracteristicas 'ESTADO B LO3', ESTADO B LC3 y ESTADO B LT3 contiene los datos nulos.

Se reemplaza los datos nulos por la moda de cada caracteristica.

In [21]:
subset_cols=['ESTADO B LO3','ESTADO B LC3','ESTADO B LT3']
[df[col].fillna(df[col].mode().values[0], inplace=True) for col in subset_cols]
df.isnull().sum()

DISENO                 0
Gramaje                0
PLANTA                 0
GAP LO(mm)             0
GAP LT(mm)             0
Temp Horno 1           0
Temp Horno 2           0
Temp Horno 3           0
Temp Horno 4           0
Temp Horno 5           0
Temp Horno 6           0
Vel Vent Horno 1       0
Vel Vent Horno 2       0
Vel Vent Horno 3       0
Vel Vent Horno 4       0
Vel Vent Horno 5       0
Vel Vent Horno 6       0
Vel Linea              0
PESO IMPREGNADO LO1    0
PESO IMPREGNADO LC     0
PESO IMPREGNADO LT1    0
VOLATILES LO2          0
VOLATILES LC2          0
VOLATILES LT2          0
FLUIDEZ                0
ESTADO B LO3           0
ESTADO B LC3           0
ESTADO B LT3           0
dtype: int64

In [22]:
max(df.isnull().sum(axis=1))

0

Se puede observar que las variables del 25 al 27 tienen 172 registros igual que las demás, también se observan que ya no tienen valores nulos.

In [24]:
df.to_csv('proyecto_pre-procesado.csv', index=False)